**SA433 &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2024**

# Lesson 19. Combining Data in Pandas

## Overview

- Data analysis rarely involves only a single data table or DataFrame


- Typically, we have many data tables, and we must combine them to answer the questions we're interested in


- In this lesson, we'll learn about two fundamental ways of combining data: *concatenation* and *merging*

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## A bunch of small example datasets 

* Let's start by importing Pandas:

In [ ]:
import pandas as pd

- We'll use the following datasets throughout this lesson:

In [ ]:
df1 = pd.read_csv('data/df1.csv')
df2 = pd.read_csv('data/df2.csv')
df3 = pd.read_csv('data/df3.csv')

left1 = pd.read_csv('data/left1.csv')
right1 = pd.read_csv('data/right1.csv')

left2 = pd.read_csv('data/left2.csv')
right2 = pd.read_csv('data/right2.csv')

left3 = pd.read_csv('data/left3.csv')
right3 = pd.read_csv('data/right3.csv')

left4 = pd.read_csv('data/left4.csv')
right4 = pd.read_csv('data/right4.csv')

left5 = pd.read_csv('data/left5.csv')
right5 = pd.read_csv('data/right5.csv')

- The cell below defines a function `display_side_by_side()` that allows us to display DataFrames side-by-side in a Jupyter notebook
    - This will let us visually examine what's going on throughout this lesson a bit more easily

In [ ]:
from IPython.display import display_html

def display_side_by_side(*dfs):
    """
    Display DataFrames side-by-side
    """
    html_str = ''.join([df.to_html() for df in dfs])
    display_html(
        html_str.replace(
            'table', 
            'table style="display:inline;margin-right:20px"'
        ), 
        raw=True
    )

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Concatenation

- Suppose we have data in which the rows/observations are spread across multiple DataFrames


- We can combine these DataFrames using `pd.concat()`


- In its most basic form, `pd.concat()` takes a list of DataFrames as input, and returns a single DataFrame that combines the rows of the input DataFrames


- For example, let's take a look at DataFrames `df1` and `df2`:

In [ ]:
display_side_by_side(df1, df2)

- We can combine the rows of `df1` and `df2` into a single DataFrame like this:

- Note that without `.reset_index()`, each row no longer has a unique index, which might be undesirable


- We can even concatenate DataFrames whose columns don't perfectly match


- For example, consider `df3`:

In [ ]:
display_side_by_side(df1, df2, df3)

- Pandas will intelligently combine `df3` with `df1` and `df2` by default:

- `pd.concat()` can be used to combine DataFrames in many other ways


- [Here's the documentation for `pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Merging

### The basics

- Another way to combine two DataFrames is to match rows based on the values of a **key** column common to both DataFrames


- This is called **merging** or **joining**

- In Pandas, we can use the `.merge()` DataFrame method to accomplish these tasks:

    ```python
    left.merge(right, on=KEYS, how=MERGE_METHOD)
     
    ``` 
    
    - `left` and `right` are the DataFrames we want to merge
    
    - `KEYS` is a column name or a *list* of column names we want to use to match rows from `left` and `right`
    
    - `MERGE_METHOD` specifies the **merge method**: `'left'`, `'right'`, `'outer'`, or `'inner'`
        - We'll go over these soon
        - By default, `how='inner'`

- [Here is the documentation for `.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)


- To illustrate, let's consider the DataFrames `left1` and `right1`:

In [ ]:
display_side_by_side(left1, right1)

- We can join the columns of `right1` with those of `left1`, matching rows by the value of `key`, like this:

- By default, the merge method is `'inner'`: the rows corresponding to the *intersection* of the keys from both DataFrames are included in the resulting DataFrame

- Summary of merge methods for the `how=...` keyword argument:

| Merge method | Description |
| :- | :- |
| `left` | Use keys from left DataFrame only |
| `right` | Use keys from right DataFrame only |
| `outer` | Use union of keys from both DataFrames |
| `inner` | Use intersection of keys from both DataFrames |

- If a key does not appear in either left or right table, the value in the merged table will be `NA`

❓ **Exercise 1.** Before running the code below, what does the resulting DataFrame look like?

In [ ]:
left1.merge(right1, on='key', how='left')

❓ **Exercise 2.** Before running the code below, what does the resulting DataFrame look like?

In [ ]:
left1.merge(right1, on='key', how='right')

❓ **Exercise 3.** Before running the code below, what does the resulting DataFrame look like?

In [ ]:
left1.merge(right1, on='key', how='outer')

### Multiple keys

- As mentioned above, the `on=...` keyword argument of `.merge()` can take a *list* of column names


- For example, consider the DataFrames `left2` and `right2`:

In [ ]:
display_side_by_side(left2, right2)

- We can merge `left2` and `right2` on the *combination* of `key1` and `key2` as follows:

### One-to-one, one-to-many, many-to-many joins

- So far, we've only seen examples of **one-to-one joins**: the keys in both the left and right DataFrames are unique


- In a **one-to-many join**, the keys in one DataFrame are unique, while the keys in the other DataFrame are possibly duplicated


- For example, let's consider `left3` and `right3`:

In [ ]:
display_side_by_side(left3, right3)

- If we merge `left3` and `right3` on `key`, we get:

- Note that the rows in `right3` are now repeated in the merged DataFrame

- In a **many-to-many join**, the keys in both the left and right DataFrames are possibly duplicated


- For example, let's consider `left4` and `right4`:

In [ ]:
display_side_by_side(left4, right4)

- If we merge `left4` and `right4` on `key`, we get:

- Take a close look at the rows with `key` equal to `K0`


- In the merged data set, we get *all* possible combinations of the rows from `left4` and `right4` with key equal to `K0`

### Common column names

- Sometimes we want to merge two DataFrames that have common column names (other than the key)


- For example, let's take a look at `left5` and `right5`, which both have a column named `A`:

In [ ]:
display_side_by_side(left5, right5)

- What happens when we merge `left5` and `right5` on `key`?

- Pandas handles the conflict in column names by appending `_x` to the left column names, and `_y` to the right column names


- You can customize this behavior with the `suffixes=(LEFT_SUFFIX, RIGHT_SUFFIX)` keyword argument in `.merge()`, like this:

- You can also use `.rename()` to clean up the column names afterwards in whatever way you wish

### Different keys for the left and right DataFrames

- We can also specify different column names as keys for the left and right DataFrames, with the `left_on=...` and `right_on=...` keyword arguments of `.merge()`


- For example, let's consider `left2` and `right2` again:

In [ ]:
display_side_by_side(left2, right2)

- We can merge `left2` and `right2`, using `key1` as the key for `left2`, and `key2` as the key for `right2`, like this:

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

In the same folder as this notebook, there are 5 CSV files that together, give a comprehensive picture of the outbound flights from NYC airports in 2013:

| Suggested DataFrame name | File | Description |
| :- | :- | :- |
| `flights` | `data/nycflights13_flights.csv` | Information about each flight &mdash; this is the same dataset we've worked with in previous lessons |
| `airlines` | `data/nycflights13_airlines.csv` | Full carrier names and their corresponding abbreviated codes |
| `airports` | `data/nycflights13_airports.csv` | Information about each airport, identified by the `faa` airport code |
| `planes` | `data/nycflights13_planes.csv` | Information about each plane, identified by its `tailnum` |
| `weather` | `data/nycflights13_weather.csv` | Weather information at each NYC airport for each hour |


You'll use these datasets in the problems below. 

In addition, you'll also use Altair to create some visualizations based on these datasets, so import Altair, and enable the VegaFusion data transformer, since these datasets are large.

In [ ]:
import altair as alt

alt.data_transformers.enable('vegafusion')

### Problem 1

Read the 5 CSV files into DataFrames, using the suggested names above. Inspect them to get a sense of their contents. Note that:

- `flights` connects to `planes` through the variable `tailnum`
- `flights` connects to `airlines` through the variable `carrier`
- `flights` connects to `airports` in two ways: the variables `origin` and `dest`
- `flights` connects to `weather` through the variables `origin`, `year`, `month`, `day`, and `hour`

### Problem 2

Merge the DataFrame `flights` and `airlines` to create a new DataFrame with the same rows as `flights`, and includes the carrier's name (`name` in `airlines`) in each row.

### Problem 3

The column `precip` in `weather` contains the precipitation in inches, for each `origin` airport, `year`, `month`, `day`, and `hour`.

Create a new DataFrame as follows. In `flights`, create a new variable containing the scheduled departure hour for each flight. Merge the DataFrames `flights` and `weather` to create a new DataFrame with the same rows as `flights`, and includes the amount of precipitation at the origin airport and the scheduled departure hour for each flight. Filter the rows in your resulting DataFrame for flights in December 2013. Drop and rename columns as necessary. 

Use Altair with the resulting DataFrame to create a scatter plot showing the relationship between the precipitation and departure delay among flights departing NYC airports in December 2013. Do you see what you expected?

*Hint.* You may find [this list of NumPy mathematical functions](https://numpy.org/doc/stable/reference/routines.math.html) useful when computing the scheduled departure hour of each flight. Also see Lesson 15.

### Problem 4

The column `year` in `planes` contains the year of manufacture for each plane.

Create a new DataFrame as follows. Merge the DataFrames `flights` and `planes` to create a new DataFrame with the same rows as `flights`, and includes the plane's year of manufacture in each row. Drop and rename columns as necessary. Then group the flights by the plane's year of manufacture and compute the average arrival delay for each group. You should end up with a DataFrame with 2 columns: the year of manufacture, and the corresponding average arrival delay.

Use Altair with the resulting DataFrame to create a line plot showing the relationship between the age of a plane and its delays among flights departing NYC airports in 2013. Do you see what you expected?

### Problem 5

Using the DataFrame `flights`, compute the number of flights from NYC airports to each destination in 2013. Merge the resulting DataFrame with `airports` so that it includes the latitude and longitude coordinates for each destination.

Use Altair with the resulting DataFrame to produce a map of the United States, representing each destination with a circle whose size is proportional to the number of flights flown there from NYC airports in 2013. In the same folder as this notebook, `data/counties_10m.json` is a TopoJSON file that contains the layers `counties`, `states`, and `nation` for the United States, based on the US Census Bureau's cartographic data. Don't forget to import GeoPandas!

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Merge, join, concatenate, and compare](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

- Lesson and problems inspired by Chapter 13 of [R for Data Science](https://r4ds.had.co.nz/)    